<div class="alert alert-block alert-success">
<b>This simulation runs a circuit with a varying number of gates for the purpose of comparing different ansatz of the wave function in order to calculate the ground state energy of the H2 molecule. </b>
<div>Obs. 1: The data is saved in external files so that it can be worked on without incurring new costs.</div>
<div>Obs. 2: For new simulation parameters, save in a file with a different name.</div>
</div>

# Instructions if you are running outside AWS cloud

1. Install aws-cli:

`sudo apt install aws-cli`

2. Setup aws credentials:

`aws configure --profile "default"`

3. Create virtual environment 

`python -m venv /path/to/new/virtual/environment`

4. Activate virtual environment

`source /path/to/new/virtual/environment/bin/activate`

# **Comparação de Diferentes Ansatz para o Cálculo da Energia do Estado Fundamental da Molécula de Hidrogênio**

## **1. Introdução**

Na simulação de sistemas quânticos utilizando **computação quântica variacional (VQE - Variational Quantum Eigensolver)**, a escolha do **ansatz** desempenha um papel fundamental na precisão dos resultados. O ansatz define a estrutura da função de onda e impacta diretamente a convergência e a qualidade da solução.

Neste notebook, comparamos diferentes ansätze para calcular a energia do estado fundamental da molécula de hidrogênio (H₂) utilizando métodos de VQE. Entre os ansätze analisados, estão:

- **Hartree-Fock (HF)**
- **UCCSD (Unitary Coupled Cluster Single and Double excitations)**
- **Hardware-Efficient Ansatz**
- **Heuristic Ansatz (RY, RZ, etc.)**

Nosso objetivo é avaliar qual ansatz fornece a melhor aproximação da energia exata com um número reduzido de portas e qubits.

---

## **2. Teoria do Método VQE**

O **Variational Quantum Eigensolver (VQE)** é um algoritmo híbrido clássico-quântico utilizado para encontrar a menor energia de um Hamiltoniano quântico. Ele combina:

1. **Um ansatz parametrizado**, que define a forma da função de onda.
2. **Um circuito quântico**, no qual os parâmetros do ansatz são otimizados para minimizar a energia.
3. **Um otimizador clássico**, que ajusta os parâmetros para minimizar o valor esperado do Hamiltoniano.

A energia é obtida como:
$$
E(\theta) = \langle \psi(\theta) | H | \psi(\theta) \rangle
$$

onde $ \theta $ são os parâmetros do circuito.

---

## **3. Definição do Problema: Molécula de H₂**

A molécula de hidrogênio é um sistema ideal para testar diferentes ansätze, pois é simples, mas ainda exibe correlação eletrônica relevante. O Hamiltoniano eletrônico pode ser expresso como:

$$
H = c_0 I + c_1 Z_0 + c_2 Z_1 + c_3 Z_0 Z_1 + c_4 X_0 X_1 + c_5 Y_0 Y_1
$$

onde os coeficientes $ c_i $ são obtidos através de cálculos de química quântica clássica.

---

## **4. Comparação dos Ansatz**

### **4.1 Hartree-Fock Ansatz**
- Utiliza apenas estados de referência baseados na aproximação de Hartree-Fock.
- Baixa complexidade, mas pode não capturar correlação eletrônica adequadamente.

### **4.2 UCCSD (Unitary Coupled Cluster Single and Double excitations)**
- Considerado um dos ansätze mais precisos para química quântica.
- Implementado com portas exponenciais de excitação.
- Custo computacional elevado devido ao grande número de portas.

### **4.3 Hardware-Efficient Ansatz**
- Usa portas quânticas genéricas (RX, RY, CNOT) distribuídas de forma eficiente para hardware quântico.
- Fácil implementação, mas pode não garantir eficiência quântica para sistemas complexos.

### **4.4 Heuristic Ansatz (RY, RZ)**
- Usa rotações parametrizadas **RY** e **RZ**.
- Baixa profundidade de circuito, mas pode exigir otimização mais refinada.

---

## **6. Conclusão**

A escolha do ansatz tem um impacto significativo na precisão e eficiência computacional do método VQE. Os principais resultados observados são:

- **Hartree-Fock**: Boa aproximação inicial, mas sem correlação eletrônica.
- **UCCSD**: Alta precisão, mas exige circuitos profundos.
- **Hardware-Efficient**: Boa alternativa para dispositivos ruidosos.
- **Heuristic Ansatz (RY, RZ)**: Alternativa leve e eficiente para cálculos aproximados.

Para a molécula de H₂, **UCCSD** fornece os melhores resultados, mas ansätze heurísticos podem ser preferíveis para execução em hardware real.

### **Referências**
- McArdle, S., et al. "Quantum computational chemistry." *Rev. Mod. Phys.*, 2020.
- Kandala, A., et al. "Hardware-efficient variational quantum eigensolver for small molecules and quantum magnets." *Nature*, 2017.
- Qiskit Textbook: https://qiskit.org/textbook/

### AWS login

In [2]:
from boto3 import Session
from braket.aws import AwsSession

# Start Boto3 Session with your credentials profile
boto_session = Session(profile_name="default")

# Initialize Braket session with Boto3 Session credentials
aws_session = AwsSession(boto_session=boto_session)

In [4]:
# Atualização de dependencias
!pip freeze > requirements.txt

# Importando Backends - Computadores Quânticos

In [5]:
from braket.aws import AwsDevice

# Recupera os dispositivos disponíveis
devices = AwsDevice.get_devices()

print("Dispositivos disponíveis na AWS Braket:")
for idx, device in enumerate(devices):
    print(f"{idx}: Nome: {device.name}, Tipo: {device.type}, ARN: {device.arn}")

Dispositivos disponíveis na AWS Braket:
0: Nome: Ankaa-3, Tipo: AwsDeviceType.QPU, ARN: arn:aws:braket:us-west-1::device/qpu/rigetti/Ankaa-3
1: Nome: Aquila, Tipo: AwsDeviceType.QPU, ARN: arn:aws:braket:us-east-1::device/qpu/quera/Aquila
2: Nome: Aria 1, Tipo: AwsDeviceType.QPU, ARN: arn:aws:braket:us-east-1::device/qpu/ionq/Aria-1
3: Nome: Aria 2, Tipo: AwsDeviceType.QPU, ARN: arn:aws:braket:us-east-1::device/qpu/ionq/Aria-2
4: Nome: Forte 1, Tipo: AwsDeviceType.QPU, ARN: arn:aws:braket:us-east-1::device/qpu/ionq/Forte-1
5: Nome: Garnet, Tipo: AwsDeviceType.QPU, ARN: arn:aws:braket:eu-north-1::device/qpu/iqm/Garnet
6: Nome: SV1, Tipo: AwsDeviceType.SIMULATOR, ARN: arn:aws:braket:::device/quantum-simulator/amazon/sv1
7: Nome: TN1, Tipo: AwsDeviceType.SIMULATOR, ARN: arn:aws:braket:::device/quantum-simulator/amazon/tn1
8: Nome: dm1, Tipo: AwsDeviceType.SIMULATOR, ARN: arn:aws:braket:::device/quantum-simulator/amazon/dm1


In [6]:
# Selecionar um dispositivo pelo índice da lista
device_index = int(input("Selecione o índice do dispositivo desejado: "))
selected_device = devices[device_index]

print(f"Dispositivo selecionado: {selected_device.name}, ARN: {selected_device.arn}")

Selecione o índice do dispositivo desejado:  6


Dispositivo selecionado: SV1, ARN: arn:aws:braket:::device/quantum-simulator/amazon/sv1


# Importing modules

In [ ]:
# Executar o modelo para H2
test_H2 = H2Model()
energy_H2 = test_H2.run_vqe()
print(f"Energia do estado fundamental do H2: {energy_H2:.6f} Ha")